In [3]:
from pathlib import Path

import polars as pl

from stocksense.database_handler import DatabaseHandler

REPORT_DIR = Path("../reports")

DATE = "2023-06-01"

In [5]:
stock_info = DatabaseHandler().fetch_stock()
df = pl.read_csv(REPORT_DIR / f"report_{DATE}.csv")
df = df.join(stock_info, on="tic", how="left")
df = df.select(pl.col("tic", "score", "name", "sector", "freturn", "adj_freturn"))
df

2024-11-25 15:15:00.626 | SUCCESS  | stocksense.database_handler.schema:create_tables:122 - Tables created successfully


tic,score,name,sector,freturn,adj_freturn
str,f64,str,str,f64,f64
"""PARA""",0.762556,"""Paramount Global""","""Communication Services""",-21.502723,-46.531975
"""FICO""",0.691881,"""Fair Isaac""","""Information Technology""",62.615358,37.586106
"""KEY""",0.6825847,"""KeyCorp""","""Financials""",59.400882,34.37163
"""FSLR""",0.668008,"""First Solar""","""Information Technology""",30.190669,5.161417
"""MPWR""",0.654031,"""Monolithic Power Systems""","""Information Technology""",48.461755,23.432502
…,…,…,…,…,…
"""DUK""",0.148876,"""Duke Energy""","""Utilities""",22.819521,-2.209732
"""PPL""",0.1465794,"""PPL Corporation""","""Utilities""",16.684222,-8.34503
"""ED""",0.137104,"""Consolidated Edison""","""Utilities""",6.764106,-18.265146


In [11]:
n = 15
top = df.head(n)
bottom = df.tail(n)

# Calculate average returns for top stocks
top_freturn = top.select(pl.col("freturn")).mean().item()
top_adj_freturn = top.select(pl.col("adj_freturn")).mean().item()

# Calculate average returns for bottom stocks
bottom_freturn = bottom.select(pl.col("freturn")).mean().item()
bottom_adj_freturn = bottom.select(pl.col("adj_freturn")).mean().item()

print(f"\nDATE {DATE}")
print(f"\nTop {n} stocks:")
print(f"Average freturn: {top_freturn:.2f}%")
print(f"Average adj_freturn: {top_adj_freturn:.2f}%")
print(f"\nBottom {n} stocks:")
print(f"Average freturn: {bottom_freturn:.2f}%")
print(f"Average adj_freturn: {bottom_adj_freturn:.2f}%")


DATE 2023-06-01

Top 15 stocks:
Average freturn: 33.63%
Average adj_freturn: 8.60%

Bottom 15 stocks:
Average freturn: 7.26%
Average adj_freturn: -17.77%
